# CDP: Unlocking Climate Solutions

####  By Gilad Shtern 27-Nov-2020

### Introduction
CDP is a global non-profit that drives companies and governments to reduce their greenhouse gas emissions, safeguard water resources, and protect forests. Each year, CDP takes the information supplied in its annual reporting process and scores companies and cities based on their journey through disclosure and towards environmental leadership.

CDP houses the world’s largest, most comprehensive dataset on environmental action. As the data grows to include thousands more companies and cities each year, there is increasing potential for the data to be utilized in impactful ways. Because of this potential, CDP is excited to launch an analytics challenge for the Kaggle community. Data scientists will scour environmental information provided to CDP by disclosing companies and cities, searching for solutions to our most pressing problems related to climate change, water security, deforestation, and social inequity.

How do you help cities adapt to a rapidly changing climate amidst a global pandemic, but do it in a way that is socially equitable?

What are the projects that can be invested in that will help pull cities out of a recession, mitigate climate issues, but not perpetuate racial/social inequities?

What are the practical and actionable points where city and corporate ambition join, i.e. where do cities have problems that corporations affected by those problems could solve, and vice versa?

How can we measure the intersection between environmental risks and social equity, as a contributor to resiliency?

### PROBLEM STATEMENT
Develop a methodology for calculating key performance indicators (KPIs) that relate to the environmental and social issues that are discussed in the CDP survey data. Leverage external data sources and thoroughly discuss the intersection between environmental issues and social issues. Mine information to create automated insight generation demonstrating whether city and corporate ambitions take these factors into account.



## Part1 - Data Exploration for Cities Responses Questionnaires

In [ ]:
#Step1 - Aggregate csv by csv subject
import pandas as pd
import os
import nbconvert
import seaborn as sns
import warnings;
warnings.filterwarnings('ignore');

path = '/kaggle/working/'
folderList = ['../input/cdp-unlocking-climate-solutions/Cities/Cities Responses',
'../input/cdp-unlocking-climate-solutions/Corporations/Corporations Responses/Climate Change',
'../input/cdp-unlocking-climate-solutions/Corporations/Corporations Responses/Water Security']
for i, valueA in enumerate(folderList):
    filesList = os.listdir(valueA)[0:3]
    for j, valueB in enumerate(filesList):
        df = pd.read_csv(valueA + "/" + valueB)
        if j == 0:
            df0 = df
        else:
            #df0 = pd.concat([df0,df], axis=1)
            df0 = df0.append(df)

    if len(valueA.split("/")) == 5:
        newFilePath = path + valueA.split("/")[4] + ".csv"
    else:
        newFilePath = path + valueA.split("/")[4] + " " + valueA.split("/")[5].split(" ")[0] + ".csv"
    df0.to_csv(newFilePath , index=False, sep=',', header=True)

In [ ]:
#Step2 - Load cities aggregate responses
Cities_Responses_df = pd.read_csv('/kaggle/working/Cities Responses.csv')
Cities_Responses_df.head()

In [ ]:
#Step3 - Check col types, NA, unique.
df = pd.DataFrame(columns = ['Col', 'Type', 'NA', '%NA', 'UniqLen']) 
colList = list(Cities_Responses_df)

for i, value in enumerate(colList):
    df.loc[i] = [value, Cities_Responses_df.dtypes[i], Cities_Responses_df[value].isna().sum(),  Cities_Responses_df[value].isna().sum()/len(Cities_Responses_df), len(Cities_Responses_df[value].unique())]

df

In [ ]:
#Step4 - Handle column
#Convert Last Update into year & month
Cities_Responses_df['ResYear'] = int(Cities_Responses_df['Last update'][1].split("/")[2].split(" ")[0])
Cities_Responses_df['ResMonth'] = int(Cities_Responses_df['Last update'][1].split("/")[1])

#Drop NA cols
Cities_Responses_df = Cities_Responses_df.drop(['Questionnaire', 'Parent Section', 'Comments', 'File Name', 'Column Name','Row Name', 'Last update'], axis=1)
#Drop specific row in which Response Answer col is NaN
Cities_Responses_df = Cities_Responses_df.dropna(subset = ['Response Answer'])

#Convert col type:int64 -> int 32
colList = list(Cities_Responses_df)
for i, value in enumerate(colList):
    if Cities_Responses_df.dtypes[i] == 'int64' or Cities_Responses_df.dtypes[i] == 'float64':
        Cities_Responses_df[value] = Cities_Responses_df[value].astype('int32')

Cities_Responses_df.head()

### Part2 - Analyze cities questionnaires to find KPIs questions 

In this part, I will try to analyze cities responses questionnaires to find which question are most likely to be related with climate issues by using NLP. at the end of this part, we can assume to keep with clean dataset.

In [ ]:
#Step5 - Question NLP check
#Create empty DF: Question Number, Question Name, Question Keywords, Climate Keywords
cities_keywords_df = pd.DataFrame(columns=['Question Number', 'Question Name', 'Question Keywords', 'Climate Keywords'])

#Keep unique questions for NLP check
cities_keywords_df['Question Number'] = Cities_Responses_df['Question Number']
cities_keywords_df['Question Name'] = Cities_Responses_df['Question Name']
cities_keywords_df['Question Keywords'] = ''
cities_keywords_df['Climate Keywords'] = ''

cities_keywords_df = cities_keywords_df.drop_duplicates(subset=['Question Number', 'Question Name'], keep="first")
cities_keywords_df.head()

#NLP
from nltk.tokenize import sent_tokenize
import nltk
en_stop = set(nltk.corpus.stopwords.words('english'))
from nltk.stem import WordNetLemmatizer
import re

stemmer = WordNetLemmatizer()
#Preprocess func
def preprocess_text(document):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(document))
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    # Converting to Lowercase
    document = document.lower()
    # Lemmatization
    tokens = document.split()
    tokens = [stemmer.lemmatize(word) for word in tokens]
    tokens = [word for word in tokens if word not in en_stop]
    tokens = [word for word in tokens if len(word) > 3]
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text

for i, value in enumerate(cities_keywords_df['Question Name']):
    text = sent_tokenize(str(value))
    cities_keywords_df['Question Keywords'].iloc[i] = preprocess_text(text)
    
cities_keywords_df.head()

#Create list of climate/pollusion dictionary
climate_change_keywords = ['carbon', 'dioxide', 'greenhouse', 'ghg', 'co2', 'emissions', 'extreme weather', 'global warming', 'climate change', 'fossil fuels','sea-level rise', 'global average temperature', 'renewable energy', 'methane', 'atmosphere', 'melt', 'waste', 'smoke', 'smog', 'sulfur', 'nitrates', 'harmful', 'hazard', 'fire', 'damage', 'acid']

#Check if col keyword contains climate/Greenhouse  keywords
for i in range(0, len(cities_keywords_df)):
    question_keywords = list(cities_keywords_df['Question Keywords'].iloc[i].split(" "))
    counter = 0
    for j, value in enumerate(question_keywords):
        try: 
            climate_change_keywords.index(value)
            counter += 1
            cities_keywords_df['Climate Keywords'].iloc[i] = counter
            
        except:
            {}

cities_keywords_df.to_csv('/kaggle/working/cities_keywords_df.csv', index=False)
cities_keywords_df.head()

In [ ]:
#Step6 - find Cities KPI impact questions
cities_keywords_df['Climate Keywords'] = pd.to_numeric(cities_keywords_df['Climate Keywords'])
cities_keywords_df['Climate Keywords'].hist()

Upon the above analysis, we can distinguish between question that contain less than at least 2 climate/greenhouse keywords with those that have more than 2 climate greenhouse keywords.
Hence, we should remove all non significant questions from the cities aggregate responses.

In [ ]:
#Step7 - Clean Cities Responses dataset
#Keep question numbers list for remove
remove_question_df = cities_keywords_df[cities_keywords_df['Climate Keywords'] <2]
remove_question_list = list(remove_question_df['Question Number'])

for i, value in enumerate(remove_question_list):
    Cities_Responses_df = Cities_Responses_df[Cities_Responses_df['Question Number'] != value]
    
Cities_Responses_df.to_csv('/kaggle/working/cities_responses_clean_df.csv', index=False)

### Part3 - Analyze climate impact keywords in matrix answers

Now, that we left with clean dataset in which contains climate KPI questions. We need to allocate climate impact answers.
However, our answers are build in a non formal matrix way. In regular rectangle matrix, our celss would be N*M. But on this questionnaires asnwers could be found with a diferent row number per a column; col A may have 3 rows, while col D may have 5 rows.
Let raise up our assumptions:
-  Assumption1: Most cities answer won't have any climate keywords.
-  Assumption2: By, contrast, sensitive climate cities will have climate keywords.

Once again, I will use NLP analysis.

In [ ]:
#Step8 - Clean answers for calc answer matrix
Cities_Responses_df = pd.read_csv('/kaggle/working/cities_responses_clean_df.csv')

#Add col clean text
Cities_Responses_df['Answer Keywords'] = ''

#Insert clean text into col
for i in range(0, len(Cities_Responses_df)):
    text = sent_tokenize(Cities_Responses_df['Response Answer'].iloc[i])
    Cities_Responses_df['Answer Keywords'].iloc[i] = preprocess_text(text)


In [ ]:
#Step9 - Calc most climate impact answer cells
#Create answer keywords col
Cities_Responses_df['Climate Keywords'] = ''

#Create list of climate/pollusion keywords
climate_change_keywords = ['carbon', 'dioxide', 'greenhouse', 'ghg', 'co2', 'emissions', 'extreme weather', 'global warming', 'climate change', 'fossil fuels','sea-level rise', 'global average temperature', 'renewable energy', 'methane', 'atmosphere', 'melt', 'waste', 'smoke', 'smog', 'sulfur', 'nitrates', 'harmful', 'hazard', 'fire', 'damage', 'acid']

#Check if col keyword contains climate/pollusion/Greenhouse
for i in range(0, len(Cities_Responses_df)):
    question_keywords = list(str(Cities_Responses_df['Answer Keywords'].iloc[i]).split(" "))
    counter = 0
    for j, value in enumerate(question_keywords):
        try:
            climate_change_keywords.index(value)
            counter += 1
        except:
            {}
    Cities_Responses_df['Climate Keywords'].iloc[i] = counter

Cities_Responses_df = Cities_Responses_df.drop(['Question Name', 'Response Answer', 'Answer Keywords'], axis=1)
Cities_Responses_df['FullAnswerID'] = Cities_Responses_df['Question Number'] + "." + Cities_Responses_df['Column Number'].map(str) + "." + Cities_Responses_df['Row Number'].map(str)
Cities_Responses_df.to_csv('/kaggle/working/cities_responses_clean_df1.csv', index=False)

In [ ]:
#Step10 - Display year trend by region as factor climate keywords
Cities_Responses_clean_df = pd.read_csv('/kaggle/working/cities_responses_clean_df1.csv')
Cities_Responses_orig_df = pd.read_csv('/kaggle/working/Cities Responses.csv')
RegionClimateByYearDF = pd.DataFrame(columns=['CDP Region', 'Year', 'Climate Ratio'])
CDPRegionList = set(Cities_Responses_clean_df['CDP Region'])
YearList = [2018, 2019, 2020]
k = 0
for i, valueA in enumerate(CDPRegionList):
    for j, valueB in enumerate(YearList):
        df0 = Cities_Responses_orig_df[Cities_Responses_orig_df['CDP Region'] == valueA]
        df1 = Cities_Responses_clean_df[Cities_Responses_clean_df['CDP Region'] == valueA]
        df0 = df0[df0['Year Reported to CDP'] == valueB]
        df1 = df1[df1['Year Reported to CDP'] == valueB]
        RegionClimateByYearDF.loc[k] = [valueA, valueB, int(df1['Climate Keywords'].sum()/len(df0) * 100)]
        k += 1

sns.catplot(x="Year", y="Climate Ratio", hue="CDP Region", kind="bar", data=RegionClimateByYearDF)

In the above graph, we see climate ratio trend by CDP regions & years. As example, Southeast Asis & Oceania climate keyword ration was reduced fromm 8% to 3% between 2018 to 2019. Also, North America reduced from 4% to 1%.

### Part4 - Build A Model

At this point, we can build an classification problem ML model. As prepare, dataset should be convert into numeric. Then, we will compare between models & choose one.

In [ ]:
#Step11 - Dictionary
dic = {}
objList = []
colList = list(Cities_Responses_clean_df)
#Create list of all object col
for i, value in enumerate(colList):
    #Per each object col, find unique values and add into list.
    if Cities_Responses_clean_df[value].dtype == 'object':
        objList += list(Cities_Responses_clean_df[value].unique())

#Remove duplicate from list and 'nan'
objList = list(set(objList))

#Build dic with values
for i, value in enumerate(objList):
    dic[value] = (i + 3) * 4 - 1
    #Go over dic and replace strings into numeric
    Cities_Responses_clean_df= Cities_Responses_clean_df.replace(value, dic[value])

for i in range(0, len(Cities_Responses_clean_df)):
    if Cities_Responses_clean_df['Climate Keywords'].iloc[i] > 0:
        Cities_Responses_clean_df['Climate Keywords'].iloc[i] = 1

#Save dic
import pickle
file_to_write = open("/kaggle/working/dic.pickle", "wb")
pickle.dump(dic, file_to_write)
#Save df
Cities_Responses_clean_df.to_csv('/kaggle/working/cities_convert_numeric.csv', index=False)

In [ ]:
#Step12 - ML PycCaret
from pycaret.classification import *

exp1 = setup(Cities_Responses_clean_df, target = 'Climate Keywords')

In [ ]:
#Step13 - Model comparision
compare_models()

In [ ]:
#Step14 -  train logistic regression model
lgbms = create_model('lightgbm')

In [ ]:
save_experiment(experiment_name = '/kaggle/working/Exp1')

In [ ]:
# summary plot
interpret_model(lgbms)
# correlation plot
interpret_model(lgbms, plot = 'summary')

### Conclussion

To summarize the answer for develop a methode for calculate KPIs that relate to the environmental and social issues that are discussed in the CDP survey data, we van suggest the followed:
- Find KPI question from the qyestionnaires by use of NLP.
- Use Light Gradient Boosting Machine model analyze the answers. Main KPI's: row number, section & question number.

Enjoy,
Gilad